In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Masking

Using TensorFlow backend.
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\silva\AppData\Roa

In [2]:
days = 30

In [3]:
def scale(training_set):
    # Feature Scaling
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    # Creating a data structure with 'days' time-steps and 1 output
    X_train = []
    y_train = []
    for i in range(days, training_set.shape[0]):
        X_train.append(training_set_scaled[i-days:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train, sc

In [4]:
def train_LSTM(X_train, y_train):
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape = (X_train.shape[1], 1)))
    #Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs = 30, batch_size = 32)
    return model

In [5]:
def predict(dataset_train, dataset_test, model, sc):
    dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
    inputs = dataset_total[len(dataset_total) - dataset_test.shape[0] - days:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(days, inputs.shape[0]):
        X_test.append(inputs[i-days:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    print(X_test.shape)
    # (273, 'days', 1)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    return X_test, predicted_stock_price

In [6]:
def plot_results(dataset_test, predicted_stock_price):
    # Visualising the results
    plt.plot(dataset_test.values, color = 'red', label = 'Real')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted')
    plt.xticks(np.arange(0,dataset_test.shape[0],50))
    plt.xlabel('Time')
    plt.ylabel('Index')
    plt.legend()
    plt.show()

In [7]:
def impute_with_prediction(df_orig, df_imputed):
    for i in df_orig.index:
        if pd.isna(df_orig.loc[i]):
            df_orig.at[i] = df_imputed.loc[i]
    return df_orig

In [8]:
df = pd.read_csv(r'output\/CompleteDengue.csv',index_col=0)

In [ ]:
for i in range(0,30):
    df_train_complete = None
    df_test_complete = None
    for col in df.columns:       
        
        test_set = df.loc['2006-07-23':, col].fillna(-1).values #Selecting test data
        training_set = df.loc['1990-04-30':'2006-07-16', col].fillna(-1).values #Selecting training data
        training_set = training_set.reshape(-1, 1)
        X_train, y_train, sc = scale(training_set)
        model = train_LSTM(X_train, y_train)
        # Dados de treinamento
        predicted_stock_price_train = model.predict(X_train)
        predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)
        #plot_results(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), predicted_stock_price_train)
        # Dados de teste
        X_test, predicted_stock_price = predict(df.loc['1990-04-30':'2006-07-16', col].fillna(-1), df.loc['2006-07-23':, col].fillna(-1), model, sc)
        #plot_results(df.loc['2015-01-02':, col].fillna(-1), predicted_stock_price)

        vazio = np.empty(days)
        predicted_stock_price_train = np.append(vazio, predicted_stock_price_train)
        series_complete_train = impute_with_prediction(df.loc['1990-04-30':'2006-07-16', col], pd.DataFrame(index=df.loc['1990-04-30':'2006-07-16', col].index, data=predicted_stock_price_train))
        df_train_complete = pd.concat([df_train_complete, pd.DataFrame(index=df.loc['1990-04-30':'2006-07-16', col].index, data=series_complete_train, columns=[col])], axis=1, join='outer')
        df_test_complete = pd.concat([df_test_complete, pd.DataFrame(data=predicted_stock_price, index = df.loc['2006-07-23':, col].index, columns = [col])], axis=1, join='outer')
    df_train_complete.to_csv(r'output\/lstm\/CompleteDengueTrainLSTM30_'+str(i)+'.csv', index = True)
    df_test_complete.to_csv(r'output\/lstm\/CompleteDengueTestLSTM30_'+str(i)+'.csv', index = True)

Epoch 1/30
814/814 [==============================] - 10s 12ms/step - loss: 0.1411
Epoch 2/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0899
Epoch 3/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0884
Epoch 4/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0900
Epoch 5/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0892
Epoch 6/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0905A: 0s - lo
Epoch 7/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0911
Epoch 8/30
814/814 [==============================] - 2s 3ms/step - loss: 0.0876
Epoch 9/30
814/814 [==============================] - 2s 3ms/step - loss: 0.0901
Epoch 10/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0913
Epoch 11/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0885
Epoch 12/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0907
Epoch 13/30
8

814/814 [==============================] - 3s 3ms/step - loss: 0.0226
Epoch 11/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0225
Epoch 12/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0215
Epoch 13/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0205
Epoch 14/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0218
Epoch 15/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0220
Epoch 16/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0215
Epoch 17/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0200
Epoch 18/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0202
Epoch 19/30
814/814 [==============================] - 2s 3ms/step - loss: 0.0192A: 1s
Epoch 20/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0203
Epoch 21/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0188
Epoch 22/30
814/814 [==

814/814 [==============================] - 3s 4ms/step - loss: 0.0126
Epoch 20/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0140
Epoch 21/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0128
Epoch 22/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0149
Epoch 23/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0127
Epoch 24/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0137
Epoch 25/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0137
Epoch 26/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0128
Epoch 27/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0128
Epoch 28/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0120
Epoch 29/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0117
Epoch 30/30
814/814 [==============================] - 3s 3ms/step - loss: 0.0127
(361, 30, 1)
Epoch 1/30
814/

814/814 [==============================] - 3s 4ms/step - loss: 0.0121
Epoch 29/30
814/814 [==============================] - 4s 4ms/step - loss: 0.0122
Epoch 30/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0128
(361, 30, 1)
Epoch 1/30
814/814 [==============================] - 14s 17ms/step - loss: 0.0042
Epoch 2/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0039
Epoch 3/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0038
Epoch 4/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0039
Epoch 5/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0039
Epoch 6/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0038
Epoch 7/30
814/814 [==============================] - 4s 4ms/step - loss: 0.0038A: 1s
Epoch 8/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0039
Epoch 9/30
814/814 [==============================] - 3s 4ms/step - loss: 0.0038A: 0s - loss:
Epoc